# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
import PyPDF2 as ppdf
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','LesionSizeLocation.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

plotting the predefined image

In [ ]:
def plot_animal_image(ax, animal):
    PATHS=('/NAS02/Rat302/Histology/DS_45.jpg',)
    
    try:
        filePath=[path for path in PATHS if animal in path][0]
    except IndexError:
        logging.error(f'Bad Animal name ({animal}), path not defined!')
        return
    
    f=PIL.Image.open(filePath)
    f.thumbnail((500,500),PIL.Image.ANTIALIAS)
    ax.imshow(f)

---

plot the Lesion Size-Location plot

In [ ]:
def plot_lesion_size_loc(root,ax,AnimalProfile, animalList=None,**kwargs):
    
    if animalList is None:
        animalList=batch_get_animal_list(root,AnimalProfile)
        
    size=[]
    loc=[]
    animals=[]
    color=[]
    for animal in animalList:
        try:
            i=HistologyExcel(NAS_PATH,animal)
        except AssertionError as e:
            logging.warning(repr(e))
            continue
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        
        try:
            t2=i.lesion_location()
            t3=i.lesion_size()
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
            
        loc.append(t2)
        size.append(t3)
        animals.append(animal)
        _,tag=lesion_type(root,animal)
        color.append(ColorCode[tag] if tag in ColorCode else 'gray')

    
    #Plotting
    
    sc=ax.scatter(loc,size,c=color,s=8,**kwargs)
    ax.axvline(0,linestyle='--',color='k',lw=.5);
    
    ax.set_xlim([-.16,.16])
    ax.set_xticks([-.1,-.05,0,.05,.1])
    ax.set_xticklabels(['DMS','$\longleftarrow$','Mid','$\longrightarrow$','DLS'])
    ax.set_xlabel('Lesion centroid')
    ax.tick_params('x',bottom=False)
    ax.spines['bottom'].set_bounds(-.15,.15)
    
    ax.set_ylabel('Lesion size')
    ax.set_ylim([0,1])
    ax.set_yticks(np.arange(0,1.01,.1))
    ax.set_yticklabels([0,'','','','',0.5,'','','','',1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    
    
#     for i, txt in enumerate(animals):
#         ax.annotate(txt[-3:], (loc[i], size[i]))
    
    return loc, size, animals


def add_legend_for_lesion_types(ax):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markerfacecolor='xkcd:gray', marker='o',markeredgewidth=.1,
                                       markeredgecolor='xkcd:black', markersize=3, label='Late')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:black', marker='D',markeredgewidth=.1,
                                       markerfacecolor='xkcd:gray', markersize=3, label='Early')

    leg=ax.legend(handles=[r_marker,b_marker],loc=2,mode=None,ncol=1,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False,handletextpad=0,columnspacing=1)
    return leg

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    fig=plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111)
    
    profile={'Type':'Good',
             'option': ['not used', 'AsymmetricLesion'],
             'rewardType':'Progressive',
             'initialSpeed':['10'],
             'Speed':'10',
             'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                    'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab']}
    
    loc, size, animals=plot_lesion_size_loc(root,ax,profile)

---

plotting the predefined image

In [ ]:
def plot_example_image(ax):
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

#     path=os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Rat276_25.png')
    
#     f=PIL.Image.open(path)
#     f.thumbnail((500,500),PIL.Image.ANTIALIAS)
#     ax.imshow(f)

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    fig=plt.figure(figsize=(1,1))
    ax=fig.add_subplot(111)
    
    plot_example_image(ax)

---

plotting group event effect

In [ ]:
def plot_group_event_effect(root, ax, Profiles, badAnimals=None, TaskParamToPlot='% good trials',
                            nPre=5, nPost=5, dot_plot=True, **kwarg):

    if badAnimals is None:
        badAnimals=[]
    
    
    #getting the data
    _,SessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)

    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])

    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values())).T
    
    x_axis=np.arange(-nSessionPre,nSessionPost)
    x_axis[x_axis>=0]+=1
    x_axis=x_axis[nSessionPre-nPre:nSessionPre+nPost]
    
    y=np.nanpercentile(data,50,axis=1)[nSessionPre-nPre:nSessionPre+nPost]
    yerr=np.nanpercentile(data,(25,75),axis=1)[:,nSessionPre-nPre:nSessionPre+nPost]
    ax.errorbar(x_axis, y, abs(yerr-y),fmt='k-o',lw=1, zorder=5, ms=2, ecolor='k', elinewidth=1, alpha=.9, **kwarg)

    
    #plotting individual animals
    if dot_plot:
        ax.plot(x_axis,data[nSessionPre-nPre:nSessionPre+nPost,:],'o-',
                ms=.3,lw=.4,alpha=.7, zorder=1, **kwarg)

    
    ax.set_xticks(x_axis)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds(-nPre,nPost)
    ax.set_xlim([-nPre-1,nPost+1])
    
#     ax.set_ylabel(TaskParamToPlot)
#     ax.set_xlabel('Sessions relative to break / lesion')

    return data[nSessionPre-nPre:nSessionPre+nPost,:]

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
              }
    profile2={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control-AfterBreak']
              }

    #number of sessions to plot
    Profiles=(profile1,profile2,)
    TaskParamToPlot='% good trials'
    plt.close('all')
    fig=plt.figure(figsize=(10,4))
    
    ax=fig.add_subplot(111)
    a=plot_group_event_effect(root, ax, Profiles,nPre=2, nPost=2,TaskParamToPlot=TaskParamToPlot)
    ax.set_ylim([0,100])

---

Plot the definition of before after ...

In [ ]:
def plot_session_def(ax, sessionSlices= tuple()):
    eps=0
    xpos=ax.get_xticks()
    
    for sli in sessionSlices:
        assert isinstance(sli, slice), 'Bad Boy!'        
        stop=sli.stop
        if stop is None:
            stop=-1
        start=sli.start
        if start>=0:
            start+=1
#             stop+=1
        
        ax.axvspan(xmin=start+eps, xmax=stop-eps, zorder=-1, alpha=.8, ec= None,
                  color='xkcd:ivory')
        
    if len(sessionSlices)==3:
        for sli,String in zip(sessionSlices,['Before','Acute','Stable']):
            stop=sli.stop
            if stop is None:
                stop=-1
            start=sli.start
            if start>=0:
                start+=1
            
            mid=(start+stop)/2
            
            ax.text(mid,ax.get_ylim()[1],s=String,
                   fontsize='xx-small', ha='center',va='top')

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:orange'
    DMSColor='purple'
    DSColor='xkcd:green'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    BadLateRats=('Rat223','Rat231')
    NAS_PATH='/NAS02'
       

    
    #===============================================
    
    # GRID 1 PARAMS
       
    profileDLS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    profileLesions1={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS1['Tag'], *profileDMS1['Tag'], *profileDS1['Tag'])))
                     }
    
    
    #================================================
    
    # GRID 2 PARAMS

    profileLesions2={'Type':'Good',
                     'option': ['not used', 'AsymmetricLesion'],
                     'rewardType':'Progressive',
                     'initialSpeed':['10'],
                     'Speed':'10',
                     'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS',
                            'Early-Lesion_DS-NoHab','Early-Lesion_DMS-NoHab']}
    
    
    
    
    #================================================
    
    # GRID 3 PARAMS
    
    profileCtrlPreBreak={'Type':'Good',
                         'rewardType':'Progressive',
                         'initialSpeed':'10',
                         'Speed':'10',
                         'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
                        }
    profileCtrlPostBreak={'Type':'Good',
                          'rewardType':'Progressive',
                          'initialSpeed':'10',
                          'Speed':'10',
                          'Tag':['Control-AfterBreak']
                         }  
    
    ProfilesBreak=(profileCtrlPreBreak,profileCtrlPostBreak,)
    nPre1=5
    nPost1=7
    
    #================================================
    
    TaskParamToPlot1='median entrance time (sec)'     
    TaskParamToPlot2="Motor Sequence Score"
    TaskParamToPlot3="Forward Running Speed"

    #================================================
    _,SessionDict=event_detect(root, ProfilesBreak[0], ProfilesBreak[1])
    badAnimals1=[]
    for animal,[pre,post] in SessionDict.items():
        if len(post) < nPost1:
            badAnimals1.append(animal)

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(4,5.5)
    fig=plt.figure(figsize=figsize,dpi=1200)
    
        

    ##########################################
    # 1: Lesion Size Loc LATE
    gs0= fig.add_gridspec(nrows=1, ncols=2, left=0, bottom=.71, right=1, top=1)
    ax0_= fig.add_subplot(gs0[0])
    ax0= fig.add_subplot(gs0[1])
    
    plot_example_image(ax0_)
    plot_example_image(ax0)
    
    
    
    
    ##########################################
    # 2: Lesion Size Loc EARLY
    gs2= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=.35, right=.45, top=.61)
    ax2= fig.add_subplot(gs2[0])
    
    *_,animals2=loc, size, animals=plot_lesion_size_loc(root,ax2,profileLesions2,marker='D',edgecolors='none',linewidths=.1,alpha=.8)
    
    ax2.set_title(f'Early lesion ($n={len(animals2)}$)', fontsize='xx-small')
    
    
    
    
    ##########################################
    # 1: Lesion Size Loc LATE
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=.55, bottom=.35, right=1,top=.61)
    ax1= fig.add_subplot(gs1[0])
    
    *_,animals1=plot_lesion_size_loc(root,ax1,profileLesions1,marker='o',edgecolors='none',linewidths=.1,alpha=.8)
    
    ax1.set_title(f'Late lesion ($n={len(animals1)}$)', fontsize='xx-small')
    ax1.set_ylabel('')
    ax1.set_yticklabels([])

    
    
    
    ##########################################
    # 5: % ET
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=.05, right=.45,top=.25)
    ax5= fig.add_subplot(gs5[0])
    data5=plot_group_event_effect(root, ax5, ProfilesBreak, nPre=nPre1, nPost=nPost1,
                                  TaskParamToPlot=TaskParamToPlot1,badAnimals=badAnimals1,c='gray')    
    ax5.set_ylim([4,10])
    ax5.spines['left'].set_bounds(4,10)
    ax5.set_ylabel('Entrance time (s)')

    #adding the vertical line + 'Break' text
#     x_=fig.transFigure.inverted().transform(ax5.transData.transform((0,0)))[0]
#     ax5.figure.add_artist(ConnectionPatch(xyA=(x_,gs5.top), xyB=(x_,gs5.bottom), 
#                                          coordsA='figure fraction', coordsB='figure fraction',
#                                          ls=':',color='gray',lw=1,zorder=10))

#     ax5.figure.add_artist(mpl.text.Text(x=x_, y=gs5.top, text='Break', c='k',ha='center',va='bottom',fontsize='xx-small'))

#     ax5.text(x=-nPre1, y=4, s=f'$n={np.sum(np.isfinite(data5[nPre1-1,:]))}$ rats',fontsize='xx-small',va='bottom')
    
    ##########################################
    # 6: % sequential score
    gs6= fig.add_gridspec(nrows=1, ncols=1, left=.55, bottom=.05, right=1,top=.25)
    ax6= fig.add_subplot(gs6[0])
    
    data6=plot_group_event_effect(root, ax6, ProfilesBreak, nPre=nPre1, nPost=nPost1,
                                  TaskParamToPlot=TaskParamToPlot2,badAnimals=badAnimals1,c='gray')
    
    ax6.set_ylim([-.01,1.01])
    ax6.spines['left'].set_bounds(0,1)
    ax6.set_yticks([0,.2,.4,.6,.8,1])
    ax6.set_yticklabels([0,20,40,60,80,100])
    ax6.set_ylabel('% Routine')
    
    
    
    ##########################################
    # 7: speed
    gs7= fig.add_gridspec(nrows=1, ncols=1, left=0.275, bottom=-.25, right=.725,top=-.05)
    ax7= fig.add_subplot(gs7[0])
    
    data7=plot_group_event_effect(root, ax7, ProfilesBreak, nPre=nPre1, nPost=nPost1,
                                  TaskParamToPlot=TaskParamToPlot3,badAnimals=badAnimals1,c='gray')    
    ax7.set_ylim([19,121])
    ax7.spines['left'].set_bounds(20,120)
    ax7.set_ylabel('Speed (cm/s)')
    ax7.set_xlabel('Session # relative to break')
    
    
    plot_session_def(ax7, sessionSlices= (slice(-2,None),slice(0,2)))
    plot_session_def(ax7, sessionSlices= (slice(5,7),))

    #STATS
    a=np.nanmean(data7[nPre1-2:nPre1,:],axis=0)
    b=np.nanmean(data7[nPre1:nPre1+2,:],axis=0)
    _,s=bootstrapTest(a-b,10000)
    delta=np.nanmean(a)-np.nanmean(b)
    ax7.text(1.5,110,s+f'\n$\delta = {delta:.2f}$ cm/s',fontsize='xx-small',c='k',ha='center')
    
    a=np.nanmean(data7[nPre1-2:nPre1,:],axis=0)
    b=np.nanmean(data7[nPre1+5:nPre1+7,:],axis=0)
    _,s=bootstrapTest(a-b,10000)
    delta=np.nanmean(a)-np.nanmean(b)
    ax7.text(6.5,110,s+f'\n$\delta = {delta:.2f}$ cm/s',fontsize='xx-small',c='k',ha='center')
    
        
    
    
        
    
#     fig.align_ylabels([ax3,ax5])
    ############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax0_,ax0,ax2,ax1,ax5,ax6,ax7)
    OFFX=np.array([.03]*len(AXES))
    OFFY=np.array([.02]*len(AXES))
    OFFY[[0,1]]=.04
#     OFFX[[0,1,2,4,6]]=0.05
    
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LesionSizeLocation.pdf'),
                format='pdf', bbox_inches='tight')
    
    thisPath  =os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LesionSizeLocation.pdf')
    lesionPath=os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Rat276_25.pdf')
    skullPath=os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','LesionSites.pdf')
    if os.path.exists(lesionPath) and os.path.exists(skullPath):
        f1=ppdf.PdfFileReader(thisPath).getPage(0)
        f2=ppdf.PdfFileReader(skullPath).getPage(0)
        f3=ppdf.PdfFileReader(lesionPath).getPage(0)

        f1.mergeTranslatedPage(page2=f2, tx=12, ty=390, expand=False)
        f1.mergeTranslatedPage(page2=f3, tx=195, ty=393, expand=False)

        writer=ppdf.PdfFileWriter()
        writer.addPage(f1)
        with open(thisPath,'wb') as f4:
            writer.write(f4)

    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()